<a href="https://colab.research.google.com/github/kstyle2198/NLP_TIPS/blob/main/Few_Shot_Text_Classification(%EB%B6%88%EA%B7%A0%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@knowledgrator/achieve-90-results-in-few-shot-text-classification-with-just-0-1-data-6bebdec1e08f

In [2]:
!pip install datasets transformers accelerate setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a6dea3de421e34989c227ce099ce61d6f9e0c5febf89de0f7b04771fb1d39aa5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built 

# Load Dataset

In [3]:
from datasets import load_dataset

#emotion
emotion_dataset = load_dataset("dair-ai/emotion")

test_dataset = emotion_dataset['test']

classes = test_dataset.features["label"].names
print(f"class개수 : {len(classes)}")
N = 8
select_num = range(len(classes)*N)
print(f"select_num : {select_num}")
train_dataset = emotion_dataset['train'].shuffle(seed=41).select(select_num)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

class개수 : 6
select_num : range(0, 48)


Okay, right now we need to download a dataset we will use the “emotion” dataset, which contains 6 classes of different emotions that describe a text. Then we will split the dataset into test and train, and from the train, we will randomly select 48 examples, with an average of 8 examples per label.

In [4]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
classes

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [6]:
train_dataset["text"][:5]

['i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i can see a dramatic improvement in my skills on the dubied already and feel that with practice i could produce lovely work in the future',
 'i feel unwelcome in this home of mine',
 'id feel triumphant or something',
 'i was already nervous about this match but by the time i got there i was simply feeling glad to be alive after a treacherous journey through foul weather']

In [7]:
train_dataset["label"][:5]

[4, 2, 0, 1, 1]

In [8]:
test_dataset['text'][:5]

['im feeling rather rotten so im not very ambitious right now',
 'im updating my blog because i feel shitty',
 'i never make her separate from me because i don t ever want her to feel like i m ashamed with her',
 'i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived',
 'i was feeling a little vain when i did this one']

In [9]:
for idx, i in enumerate(train_dataset):
  print(i)

  if idx > 10:
    break

{'text': 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing', 'label': 4}
{'text': 'i can see a dramatic improvement in my skills on the dubied already and feel that with practice i could produce lovely work in the future', 'label': 2}
{'text': 'i feel unwelcome in this home of mine', 'label': 0}
{'text': 'id feel triumphant or something', 'label': 1}
{'text': 'i was already nervous about this match but by the time i got there i was simply feeling glad to be alive after a treacherous journey through foul weather', 'label': 1}
{'text': 'i feel deeply disappointed', 'label': 0}
{'text': 'i woke up at around am or am the next day crunched at the bed because i was feeling a terrible headache so painful i was awaken from my sleep', 'label': 0}
{'text': 'i can t tell you fortunate i feel to have access to so many wonderfully talented photogr

# SetFit

Firstly, we will see what results we can achieve with SetFit — an alternative few-shot learning approach that uses text embeddings for classification. Then, we will see how our approach is more powerful than SetFit.

In [10]:
from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report

In [11]:
model = SetFitModel.from_pretrained("BAAI/bge-base-en-v1.5")

args = TrainingArguments(
   batch_size=32,
   num_epochs=3,
)

trainer = Trainer(
   model=model,
   args=args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [12]:
trainer.train()

***** Running training *****
  Num examples = 53
  Num epochs = 3
  Total optimization steps = 159
  Total train batch size = 32


Step,Training Loss


In [13]:
preds = model.predict(test_dataset['text'])
preds

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([0, 0, 0,  ..., 1, 1, 0])

In [14]:
print(classification_report(test_dataset['label'], preds, target_names=classes, digits=4))

              precision    recall  f1-score   support

     sadness     0.5143    0.8072    0.6283       581
         joy     0.6592    0.8489    0.7421       695
        love     0.1503    0.1447    0.1474       159
       anger     0.3590    0.0509    0.0892       275
        fear     1.0000    0.0045    0.0089       224
    surprise     0.0000    0.0000    0.0000        66

    accuracy                         0.5485      2000
   macro avg     0.4471    0.3094    0.2693      2000
weighted avg     0.5518    0.5485    0.4654      2000



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python

# Comprehend-it method

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'knowledgator/comprehend_it-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [16]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import random
import torch
import evaluate
import numpy as np

In [17]:
accuracy = evaluate.load("accuracy")
accuracy

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [18]:
# 샘플 데이터를 10개씩 증강.. (원래 48개인데.. 10문장씩 복사 증강하여 총 데이터가 480개가 됨)

def transform_dataset(dataset, classes, template = '{}'):
   new_dataset = {'sources':[], 'targets': [], 'labels': []}

   texts = dataset['text']
   print(f"texts 앞에 두개: {texts}")
   labels = dataset['label']
   print(f"labels 앞에 두개: {labels}")

   label2count = {}
   for label in labels:
       if label not in label2count:
           label2count[label]=1
       else:
           label2count[label]+=1
   print(f"label2count: {label2count}")

   count = len(labels)
   print(f"count: {count}")

   label2prob = {label:lc/count for label, lc in label2count.items()}
   print(f"label2prob: {label2prob}")

   unique_labels = list(label2prob)
   print(f"unique_labels: {unique_labels}")

   probs = list(label2prob.values())
   print(f"probs: {probs}")

   ids = list(range(len(labels)))
   print(f"ids: {ids}")

   print(f"classes: {classes}")

   for text, label_id in zip(texts, labels):
       label = classes[label_id]
       for i in range(len(classes)-1):
           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(label))
           new_dataset['labels'].append(1.)

       print(f"new_dataset1: {new_dataset}")

       for i in range(len(classes)-1):
           neg_class_ = label
           while neg_class_==label:
               # neg_class_ = random.sample(classes, k=1)[0]
               neg_lbl = np.random.choice(unique_labels, p=probs)
               neg_class_ = classes[neg_lbl]

           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(neg_class_))
           new_dataset['labels'].append(-1.)
       print(f"new_dataset2: {new_dataset}")

   result = Dataset.from_dict(new_dataset)
   print(f"result : {result}")
   return result


The provided code defines a function transform_dataset that takes a dataset, a list of classes, and an optional template as input and transforms it into a new dataset. The purpose of this function appears to be to create a dataset suitable for training a model for text classification or a similar task.

Here's a breakdown of the code:

1. Input Parameters:
- dataset: Input dataset containing 'text' and 'label' keys.
- classes: List of classes or labels.
- template: A string template used to format the target label. It uses the placeholder {} which is intended to be replaced with the actual label.

2. Initialization:
- new_dataset: A dictionary with keys 'sources', 'targets', and 'labels'. It will store the transformed dataset.

3. Processing Labels:

- Count the occurrences of each label in the input dataset.
- Calculate the probability of each label occurring in the dataset.

4. Data Transformation Loop:

- For each text and label in the original dataset:
- Repeat the following steps for each class in the list of classes (excluding the last one):
Append the text to 'sources'.
Append the formatted target label (using the template) to 'targets'.
Append a label of 1.0 to 'labels'.
- Repeat the following steps for each class in the list of classes (excluding the last one):
Choose a negative class different from the current label.
Append the text to 'sources'.
Append the formatted target label (using the template) to 'targets'.
Append a label of -1.0 to 'labels'.

5. Output:

- The transformed dataset is returned using Dataset.from_dict(new_dataset).

It's worth noting that this function seems to create a dataset suitable for training a binary classification model, where the goal is to distinguish between the actual class and a randomly selected negative class. The negative class is selected with probabilities proportional to their occurrence in the original dataset. The choice of negative class is ensured to be different from the actual class. The function returns a dataset with 'sources', 'targets', and 'labels', where 'labels' are either 1.0 or -1.0, indicating whether the example corresponds to the actual class or a negative class.

In [19]:
def compute_metrics(eval_pred):
   predictions, labels = eval_pred

   predictions = np.argmax(predictions, axis=1)

   return accuracy.compute(predictions=predictions, references=labels)

In [20]:
def tokenize_and_align_label(example):
   hypothesis = example['targets']

   seq = example["sources"]+hypothesis

   tokenized_input = tokenizer(seq, truncation=True, max_length=512,
                                                    padding="max_length")

   label = example['labels']
   if label==1.0:
       label = torch.tensor(1)
   elif label==0.0:
       label = torch.tensor(2)
   else:
       label = torch.tensor(0)
   tokenized_input['label'] = label
   return tokenized_input

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DebertaV2TokenizerFast(name_or_path='knowledgator/comprehend_it-base', vocab_size=128000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, 

In [22]:
transform_dataset(train_dataset, classes)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Dataset({
    features: ['sources', 'targets', 'labels'],
    num_rows: 480
})

In [23]:
dataset = transform_dataset(train_dataset, classes)
dataset

texts 앞에 두개: ['i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing', 'i can see a dramatic improvement in my skills on the dubied already and feel that with practice i could produce lovely work in the future', 'i feel unwelcome in this home of mine', 'id feel triumphant or something', 'i was already nervous about this match but by the time i got there i was simply feeling glad to be alive after a treacherous journey through foul weather', 'i feel deeply disappointed', 'i woke up at around am or am the next day crunched at the bed because i was feeling a terrible headache so painful i was awaken from my sleep', 'i can t tell you fortunate i feel to have access to so many wonderfully talented photographers like yourself', 'i feel as though ive reached a point in my career where im highly respected there', 'i couldnt help but feel sincere g

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
train_dataset['text'][:2]

['i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i can see a dramatic improvement in my skills on the dubied already and feel that with practice i could produce lovely work in the future']

In [25]:
train_dataset['label'][:2]

[4, 2]

In [26]:
# 10개까지 복사 증강된 동일 데이터
dataset["sources"][0:11]

['i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing',
 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my fav

In [27]:
dataset["targets"][:11]

['fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'joy',
 'love',
 'sadness',
 'sadness',
 'joy',
 'love']

In [28]:
dataset["labels"][:11]

[1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0]

In [29]:
tokenized_dataset = dataset.map(tokenize_and_align_label)
tokenized_dataset

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Dataset({
    features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 480
})

In [30]:
for i in tokenized_dataset:
  print(i)
  break

{'sources': 'i was feeling so reluctant the whole day today the only thing that i feel like doing is just sticking my ass on the benches ground having heart to heart talks with my favs staring into space and nothing', 'targets': 'fear', 'labels': 1.0, 'input_ids': [1, 584, 284, 1439, 324, 12621, 262, 783, 406, 561, 262, 364, 576, 272, 584, 551, 334, 653, 269, 348, 9884, 312, 283, 268, 277, 262, 18488, 1353, 591, 924, 264, 924, 3496, 275, 312, 28821, 268, 10749, 352, 754, 263, 942, 42606, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [31]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 432
    })
    test: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 48
    })
})

In [32]:
training_args = TrainingArguments(
   output_dir='comprehendo',
   learning_rate=3e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
)

In [33]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.196233,0.916667
2,No log,0.019175,1.000000
3,No log,0.002416,1.000000


TrainOutput(global_step=162, training_loss=0.2910758595407745, metrics={'train_runtime': 184.755, 'train_samples_per_second': 7.015, 'train_steps_per_second': 0.877, 'total_flos': 341001104670720.0, 'train_loss': 0.2910758595407745, 'epoch': 3.0})

In [35]:
trainer.save_model('comprehender')

In [36]:
from transformers import pipeline
from sklearn.metrics import classification_report
from tqdm import tqdm

classifier = pipeline("zero-shot-classification", model='comprehender',tokenizer=tokenizer, device=device)

In [37]:
preds = []
label2idx = {label: id for id, label in enumerate(classes)}

for example in tqdm(test_dataset):
   pred = classifier(example['text'],classes)['labels'][0]
   idx = label2idx[pred]
   preds.append(idx)
preds[:3]

  0%|          | 10/2000 [00:01<04:09,  7.98it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [04:40<00:00,  7.13it/s]


[0, 0, 2]

In [40]:
print(classification_report(test_dataset['label'], preds,
                                        target_names=classes, digits=4))

              precision    recall  f1-score   support

     sadness     0.6408    0.6816    0.6606       581
         joy     0.7747    0.6878    0.7287       695
        love     0.3419    0.3333    0.3376       159
       anger     0.7436    0.4218    0.5383       275
        fear     0.6369    0.4464    0.5249       224
    surprise     0.1515    0.6818    0.2479        66

    accuracy                         0.5940      2000
   macro avg     0.5482    0.5421    0.5063      2000
weighted avg     0.6611    0.5940    0.6129      2000

